In [ ]:
pip install transformers

In [ ]:
pip install scipy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import re

# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

while True:
    user_text = input("\nEnter your text (or type 'exit' to quit): ")

    # Input Type Validation
    if not isinstance(user_text, str):
        print("Please enter a valid text.")
        continue

    # Check for Empty Input
    if not user_text.strip():
        print("Please enter a non-empty text.")
        continue

    if user_text.lower() == 'exit':
        break

    # preprocess user_text
    user_words = []

    for word in user_text.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        elif word.startswith('http'):
            word = "http"
        user_words.append(word)

    user_proc = " ".join(user_words)

    # sentiment analysis
    encoded_user_text = tokenizer(user_proc, return_tensors='pt')
    output = model(**encoded_user_text)

    scores = output.logits[0].detach().numpy()
    scores = softmax(scores)

    predicted_class_index = scores.argmax()
    predicted_class_label = labels[predicted_class_index]
    predicted_probabilities = scores.tolist()

    # Display the predicted sentiment probabilities and class labels
    print(f"\nPredicted Sentiment Probabilities: [Negative: {predicted_probabilities[0]}, Neutral: {predicted_probabilities[1]}, Positive: {predicted_probabilities[2]}]")
    print(f"Predicted Class: {predicted_class_label}")



Enter your text (or type 'exit' to quit): The coffee shop is known for its cozy atmosphere.

Predicted Sentiment Probabilities: [Negative: 0.0014519968535751104, Neutral: 0.10550512373447418, Positive: 0.8930428624153137]
Predicted Class: Positive

Enter your text (or type 'exit' to quit): The rainy weather ruined our plans for an outdoor picnic.

Predicted Sentiment Probabilities: [Negative: 0.9109612107276917, Neutral: 0.08155105262994766, Positive: 0.007487711030989885]
Predicted Class: Negative

Enter your text (or type 'exit' to quit): he concert last night was incredible – the band played all my favorite songs.

Predicted Sentiment Probabilities: [Negative: 0.0017945414874702692, Neutral: 0.006745282094925642, Positive: 0.9914602041244507]
Predicted Class: Positive

Enter your text (or type 'exit' to quit): exit


In [ ]:
pip install tweepy

In [ ]:
pip install pandas

In [ ]:
pip install configparser

In [ ]:
import tweepy
import configparser
import pandas as pd

# read configs
config = configparser.ConfigParser()
config.read('/content/config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

# authentication
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

public_tweets = api.home_timeline()

# create dataframe
columns = ['Time', 'User', 'Tweet']
data = []
for tweet in public_tweets:
    data.append([tweet.created_at, tweet.user.screen_name, tweet.text])

df = pd.DataFrame(data, columns=columns)

df.to_csv('tweepy1.csv')